### Simplex Tableau
---

Construção do algoritmo Simplex na forma Tableau:

* Não precisamos dos passos dados no Simplex "raiz", as iterações são compostas apenas de operações matriciais
* No Tableau, criamos uma matriz indexada pelas variábeis básicas, composta pela matriz de restrições com o vetor $b$ concatenado, e o vetor de custo com o resultado $z$ concatenado, como mostra a figura abaixo:

<img src="files/simplex_tableau.jpg">

In [1]:
import numpy as np

#### PL do exercício 2.4 (2)

In [45]:
A = np.array([[1, -1, 2, -1, 0],
              [2, 0, 1, -1, 1]])
c = np.array([1, -2, 0, 1, 3]).reshape(5,1)
b = np.array([1, -1]).reshape(2,1)

In [46]:
A

array([[ 1, -1,  2, -1,  0],
       [ 2,  0,  1, -1,  1]])

In [47]:
b

array([[ 1],
       [-1]])

In [48]:
c

array([[ 1],
       [-2],
       [ 0],
       [ 1],
       [ 3]])

In [54]:
B = np.array([0,3]) # indexação começa em 0! Equivale a B = [1, 4]

Passar o problema para a forma canônica:

In [55]:
def canonical_form(A, b, c, B=None, w=0):
    """
    Passa o PL para a forma canônica: (i) A_B deve ser identidade, (ii) c_B = 0
    """

    if np.all(B != None):
        if not np.all(c[B] != 0) or not np.all(A[:, B] != np.identity(len(B))):

            A_Binv = np.linalg.inv(A[:, B])
            y = A_Binv.T.dot(c[B])
            c = c.T - y.T.dot(A).ravel()
            c = c.T
            w = w + np.dot(y.T, b)

            A = A_Binv.dot(A)
            b = A_Binv.dot(b)

            print('A: \n', A)
            print('b: \n', b)
            print('c: \n', c)
            print('w: \n', w)
            print(">> Problema canonizado!")
            return A, b, c, w

        else:
            print(">> Problema ja na forma canonica!")
            return
    
    else:
        pass

In [56]:
A, b, c, w = canonical_form(A, b, c, B)

A: 
 [[ 1.  1. -1.  0.  1.]
 [ 0.  2. -3.  1.  1.]]
b: 
 [[-2.]
 [-3.]]
c: 
 [[ 0.]
 [-5.]
 [ 4.]
 [ 0.]
 [ 1.]]
w: 
 [[0.]]
>> Problema canonizado!


Construindo o *tableu* inicial:

In [131]:
T = np.concatenate((np.zeros(A.shape[0]).reshape(A.shape[0],1), A, b), axis=1)
T = np.concatenate((np.array([1] + list(-c.T[0]) + [0]).reshape(1, c.shape[0]+2), T), axis=0)
T

array([[ 1., -0.,  5., -4., -0., -1.,  0.],
       [ 0.,  1.,  1., -1.,  0.,  1., -2.],
       [ 0.,  0.,  2., -3.,  1.,  1., -3.]])

*Pivot* - trocando uma variável básica por uma não básica:

In [132]:
k = np.argmin(T[0, :]) # coluna com o menor valor em -c (coluna da variável não básica a ser aumentada)
T[:, k]

array([-4., -1., -3.])

In [133]:
razao = T[1:, -1]/T[1:, k] # divide os valores de b pela coluna (máximo de aumento da variável da coluna k)
razao

array([2., 1.])

In [134]:
razao = np.where(razao < 0, np.nan_to_num(np.inf), razao) # substitui valores negativo por +inf
i = np.argmin(razao) # obtem o índice do menor valor (menor dentre os máximos para a variável da coluna k)
i

1

In [135]:
T[i, k]

-1.0

In [136]:
T[i,:] = T[i,:]/T[i, k] # dividindo a linha pelo valor eacolhido
T

array([[ 1., -0.,  5., -4., -0., -1.,  0.],
       [-0., -1., -1.,  1., -0., -1.,  2.],
       [ 0.,  0.,  2., -3.,  1.,  1., -3.]])

### Exemplos
---

#### Fase I + Fase II